In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform = Reform.from_dict({
  "gov.contrib.reconciliation.medicaid_work_requirement.senate.in_effect": {
    "2027-01-01.2100-12-31": True
  },
}, country_id="us")


In [3]:
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

In [14]:
year = 2028
age = baseline.calculate("age", map_to="household", period=year)
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)
medicaid_baseline = baseline.calculate("medicaid_enrolled", map_to="household", period=year)
household_id = baseline.calculate("household_id", map_to="household", period=year)
disability = baseline.calculate("is_disabled", map_to="household", period=year)
net_income = baseline.calculate("household_net_income", map_to="household", period=year)

In [5]:
medicaid_reform = reformed.calculate("medicaid_enrolled", map_to="household", period=year)

In [6]:
medicaid_baseline.sum()/1e6

88.03708112231564

In [7]:
medicaid_reform.sum()/1e6

66.17075737452024

In [8]:
medicaid_cost_baseline = baseline.calculate("medicaid_cost", map_to="household", period=year)
medicaid_cost_reform = reformed.calculate("medicaid_cost", map_to="household", period=year)

In [9]:
medicaid_cost_baseline.sum()/1e9

754.2560939202893

In [10]:
medicaid_cost_reform.sum()/1e9

583.9976766367972

In [17]:
data = {
    "household_id": household_id,
    "age": age,
    "State": state,
    "Married": married,
    "Num_Dependents": num_dependents,
    "net_income": net_income,
    "medicaid_baseline": medicaid_baseline,
    "medicaid_reform": medicaid_reform,
    "disability": disability,
}

df_outputs = pd.DataFrame(data)
df_outputs.sample(n=10) 



,household_id,age,State,Married,Num_Dependents,net_income,medicaid_baseline,medicaid_reform,disability
25763,108937,58.0,IL,0.0,0.0,76981.750000,0.0,0.0,1.0
15052,63037,60.0,TX,1.0,3.0,66695.476562,0.0,0.0,0.0
23671,102468,138.0,PA,0.0,0.0,36383.925781,3.0,0.0,0.0
29397,125991,127.0,VA,1.0,2.0,85631.062500,0.0,0.0,0.0
35968,153372,133.0,TX,1.0,1.0,55173.074219,0.0,0.0,1.0
177,1452,168.0,NH,1.0,0.0,277564.562500,0.0,0.0,0.0
40669,174681,145.0,CA,1.0,0.0,108951.515625,0.0,0.0,0.0
6118,22615,21.0,WI,0.0,0.0,60602.281250,0.0,0.0,1.0
39910,172254,85.0,CA,0.0,0.0,72438.218750,1.0,1.0,1.0
3931,15787,145.0,OH,1.0,2.0,116171.570312,1.0,1.0,2.0


In [18]:
df_medicaid = df_outputs[df_outputs["medicaid_baseline"].gt(0)]
df_medicaid.sample(n=10) 


,household_id,age,State,Married,Num_Dependents,net_income,medicaid_baseline,medicaid_reform,disability
39714,171625,190.0,CA,1.0,0.0,266112.437500,2.0,1.0,2.0
25746,108895,162.0,IL,1.0,1.0,113722.921875,1.0,0.0,1.0
34089,145258,123.0,AR,1.0,0.0,52366.300781,1.0,0.0,1.0
21260,93908,60.0,MA,0.0,0.0,12498.601562,2.0,1.0,0.0
39141,169644,31.0,CA,0.0,1.0,20294.919922,2.0,2.0,0.0
5781,21507,87.0,MI,0.0,1.0,126903.640625,1.0,0.0,1.0
25970,109575,151.0,IL,1.0,1.0,77474.335938,2.0,1.0,0.0
35709,152511,145.0,TX,1.0,3.0,54013.031250,2.0,2.0,0.0
21664,96019,147.0,CT,0.0,0.0,105085.656250,1.0,1.0,0.0
25351,107589,52.0,IL,0.0,2.0,24615.585938,3.0,3.0,1.0
